# Deeper processing for ML analysis

Reprocessing of `*_pert.csv` data (see `prep/dataprep.ipynb` for more info).

- First we get consolidated data for all the systems (similar to reduce.py) into `proc_*_pert.vsc`
- Repeat for planets in HZ in `hz_*_pert.csv`



In [5]:
import pandas as pd
import numpy as np

In [6]:
fold='../prep/'

In [7]:
resf=['no_pert.csv','lo_pert.csv','hi_pert.csv']
for resfile in resf:
    resfile=fold+resfile
    dff=pd.read_csv(resfile)
    dff['mass']=dff['emepla(i)/emet']+dff['emegas(i)']
    nlist=['ident', 'com', 'nplanets', 'massbudget', 'massefficiency', 'sigmag0', 'md', 'rc', 'ms', 'metal', 'taugas',  'qest','ngi','mtr','apert']
    dff=dff[dff.mass<6500] # approx. 20 jupiter masses
    rdata=np.zeros((len(np.unique(dff.ident)),len(nlist)))
    kk=0
    fpert=dff.fpert.iloc[0]
    for i in np.unique(dff.ident):    
        filter=dff.ident==i
        dummy=dff[filter]
        pmass=dummy.mass
        com=((pmass*dummy['a(i)']).sum())/pmass.sum()
        npl=len(dummy)
        mtot=pmass.sum()
        sigmag0=dummy.sigmag_0.iloc[0]
        effm=mtot*3e-6/(dummy.emed.iloc[0])
        rc=dummy.rc.iloc[0]
        ms=dummy.emestar.iloc[0]/2e33
        md=dummy.emed.iloc[0]*ms
        metal=dummy.emetal.iloc[0]
        taugas=dummy.taugas.iloc[0]   
        qest=dummy.qest.iloc[0]
        boogi=dummy.mass>10 # 10 earth masses as giant mass limit
        ngi=(boogi).sum()
        apert=dummy.apert.iloc[0]
        mtr=(dummy.mass[np.logical_not(boogi)]).sum()
        rdata[kk,:]=i,com,npl,mtot,effm,sigmag0,md,rc,ms,metal,taugas,qest,ngi,mtr,apert
        kk=kk+1
    pd.DataFrame(rdata,columns=nlist).to_csv(resfile[:8]+'proc_%s.csv'%resfile[8:-4])

In [8]:
resf=['no_pert.csv','lo_pert.csv','hi_pert.csv']
for resfile in resf:
    resfile=fold+resfile
    dff=pd.read_csv(resfile)
    hzmin=0.75*(dff.emestar/2e33)**2
    hzmax=1.77*(dff.emestar/2e33)**2
    dff=dff[(dff['a(i)']>hzmin)&(dff['a(i)']<hzmax)]
    dff['mass']=dff['emepla(i)/emet']+dff['emegas(i)']
    dff=dff[dff.mass<6500] # approx. 20 jupiter masses
    nlist=['ident', 'com', 'nplanets', 'massbudget', 'massefficiency', 'sigmag0', 'md', 'rc', 'ms', 'metal', 'taugas',  'qest','ngi','mtr','apert']
    
    rdata=np.zeros((len(np.unique(dff.ident)),len(nlist)))
    kk=0
    fpert=dff.fpert.iloc[0]
    for i in np.unique(dff.ident):    
        filter=dff.ident==i
        dummy=dff[filter]
        pmass=dummy.mass
        com=((pmass*dummy['a(i)']).sum())/pmass.sum()
        npl=len(dummy)
        mtot=pmass.sum()
        sigmag0=dummy.sigmag_0.iloc[0]
        effm=mtot*3e-6/(dummy.emed.iloc[0])
        rc=dummy.rc.iloc[0]
        ms=dummy.emestar.iloc[0]/2e33
        md=dummy.emed.iloc[0]*ms
        metal=dummy.emetal.iloc[0]
        taugas=dummy.taugas.iloc[0]   
        qest=dummy.qest.iloc[0]
        boogi=dummy.mass>10 # 10 earth masses as giant mass limit
        ngi=(boogi).sum()
        apert=dummy.apert.iloc[0]
        mtr=(dummy.mass[np.logical_not(boogi)]).sum()
        rdata[kk,:]=i,com,npl,mtot,effm,sigmag0,md,rc,ms,metal,taugas,qest,ngi,mtr,apert
        kk=kk+1
    pd.DataFrame(rdata,columns=nlist).to_csv(resfile[:8]+'hz_%s.csv'%resfile[8:-4])